In [ ]:
from models import Lumina

import torch
import torchaudio
from argparse import Namespace
import numpy as np


config = Namespace(
    in_channel=1,          # Code mặc định là 1 (Mono)
    channel=512,           # Code mặc định là 512 (Lưu ý: code cũ của bạn là 128, hãy check lại file config gốc nếu size model quá lớn)
    n_res_block=2,         # (Giữ nguyên hoặc chỉnh theo config lúc train)
    n_res_channel=32,      # (Giữ nguyên hoặc chỉnh theo config lúc train)
    quantizer="fsq",       # Bắt buộc có để tránh lỗi AttributeError
    levels=[7, 5, 5, 5, 5],# Mặc định trong code của bạn. Tích = 4375
    embed_dim=5,           
    
    n_embed=4375,          # 7*5*5*5*5 = 4375
    decay=0.99
)

In [15]:
checkpoint_path = "checkpoints/audio_fsq-n_embed-4375/ckpts/epoch_195.pt" # Đường dẫn file checkpoint của bạn
input_audio_path = "/kaggle/lumina/assets/en_00002.wav"    # File âm thanh gốc muốn test
output_audio_path = input_audio_path[:-4]+"_re.wav" # File âm thanh sau khi tái tạo

target_sample_rate = 24000  
device = "cuda" if torch.cuda.is_available() else "cpu"

lumina = Lumina(
    config=config, 
    checkpoint_path=checkpoint_path, 
    sample_rate=target_sample_rate, 
    device=device
)

Loading checkpoint from: checkpoints/audio_fsq-n_embed-4375/ckpts/epoch_195.pt
Successfully loaded state dict!


In [16]:
import os

tokens = lumina.encode(input_audio_path)
print(f"\n🔊 Đang giải mã tokens ngược lại thành audio...")

recon_wav, sr = lumina.decode(tokens)

if recon_wav.dim() == 1:
    recon_wav = recon_wav.unsqueeze(0)
    
torchaudio.save(output_audio_path, recon_wav, sr)
print(f"✅ Đã lưu file tái tạo tại: {output_audio_path}")
print(f"   - Shape output wav: {recon_wav.shape}")



🔊 Đang giải mã tokens ngược lại thành audio...
✅ Đã lưu file tái tạo tại: /kaggle/lumina/assets/en_00002_re.wav
   - Shape output wav: torch.Size([1, 228352])
